In [1]:
import sys
import os

repo_root = os.path.abspath(os.path.join(os.path.dirname(os.getcwd())))
sys.path.append(repo_root)

print(repo_root)

/home/jackyeung99/classes/class_financial-econometrics


In [2]:
import matplotlib.pyplot as plt
import math 
import numpy as np
import pandas as pd
from pandas_datareader import data as pdr
from statsmodels.tsa.stattools import adfuller, coint

In [3]:
data = ['CPIAUCSL', 'GDPC1', 'CNP16OV', 'FEDFUNDS']

In [4]:
df = pdr.DataReader(
    ["GDPC1","CNP16OV","CPIAUCSL","FEDFUNDS","WM2NS"],
    "fred",
    start="1980-01-01",
    end="2025-07-01"
).astype(float)

df['CNP16OV'] =  df['CNP16OV'].astype(float) # Civilian non-instituional population 
df['POP'] = df["CNP16OV"] * 1000.0

df['GDPC1'] = df['GDPC1'].astype(float) # real gdp
df['CPIAUCSL'] = df['CPIAUCSL'].astype(float) # CPI price level
df['FEDFUNDS'] = df['FEDFUNDS'].astype(float) # Fed funds

In [5]:
df_q = (
    df
    .resample('QE') 
    .agg({
        'GDPC1': 'last',     
        'CPIAUCSL': 'mean',  
        'CNP16OV': 'mean',
        'POP': 'mean',       
        'FEDFUNDS': 'mean',  
        'WM2NS': 'mean'      
    })
    .loc[:'2025-06-30']      
).reset_index()

In [6]:
df_q["RGDP_per_capita"] = df_q["GDPC1"] / df_q["CNP16OV"]
df_q["YGR"] = 100 * (np.log(df_q["RGDP_per_capita"]) - np.log(df_q['RGDP_per_capita'].shift(-1)))
df_q['INF'] = 400 * np.log(df_q['CPIAUCSL']/ df_q['CPIAUCSL'].shift(-1))

In [7]:
df_q

,DATE,GDPC1,CPIAUCSL,CNP16OV,POP,FEDFUNDS,WM2NS,RGDP_per_capita,YGR,INF
0,1980-03-31,7341.557,79.033333,166762.333333,1.667623e+08,15.046667,NaN,0.044024,2.472968,-13.273719
1,1980-06-30,7190.289,81.700000,167415.666667,1.674157e+08,12.686667,NaN,0.042949,0.533201,-7.437563
2,1980-09-30,7181.743,83.233333,168110.666667,1.681107e+08,9.836667,NaN,0.042720,-1.501551,-11.059156
3,1980-12-31,7315.677,85.566667,168693.666667,1.686937e+08,15.853333,1601.088889,0.043367,-1.594093,-10.913261
4,1981-03-31,7459.022,87.933333,169279.000000,1.692790e+08,16.570000,1617.776923,0.044063,1.073123,-8.253904
...,...,...,...,...,...,...,...,...,...,...
177,2024-06-30,23286.508,313.095667,268250.666667,2.682507e+08,5.330000,20993.761538,0.086809,-0.594502,-1.386308
178,2024-09-30,23478.570,314.182667,268860.000000,2.688600e+08,5.263333,21107.400000,0.087326,-0.234668,-2.988338
179,2024-12-31,23586.542,316.538667,269463.333333,2.694633e+08,4.650000,21427.438462,0.087532,1.412246,-3.714732
180,2025-03-31,23548.210,319.492000,272851.666667,2.728517e+08,4.330000,21644.415385,0.086304,-0.744814,-1.634672


In [18]:
coint(df_q['RGDP_per_capita'], df_q['CPIAUCSL'])


(np.float64(-1.4024251440751654),
 np.float64(0.7966807651348102),
 array([-3.95797112, -3.37009572, -3.06796507]))

In [22]:
coint(np.log(df_q['GDPC1'].iloc[3:]), np.log(df_q['WM2NS'].iloc[3:]))


(np.float64(-1.316790854776897),
 np.float64(0.8246115704527541),
 array([-3.9590257 , -3.37067175, -3.06836281]))

In [10]:


def adf_summary(series, name, trend="c", maxlags=12):
    s = series.dropna()
    stat, p, lags, nobs, crit, *_ = adfuller(s, regression=trend, maxlag=maxlags, autolag="AIC")
    return {
        "series": name,
        "trend": trend,
        "ADF stat": stat,
        "p-value": p,
        "lags": lags,
        "nobs": nobs,
        "1%": crit["1%"], "5%": crit["5%"], "10%": crit["10%"]
    }

In [ ]:
trend_map = {
    "RGDP_per_capita": "ct",  # trending level
    "CPIAUCSL": "ct",         # trending level
    "YGR": "c",               # rate
    "INF": "c",               # rate
}

cols = ["RGDP_per_capita", "CPIAUCSL", "YGR", "INF"]
rows = [adf_summary(df_q[c], c, trend=trend_map.get(c, "c")) for c in cols]
adf_df = pd.DataFrame(rows)


adf_df_display = adf_df.copy()
adf_df_display = adf_df_display[["series","trend","ADF stat","p-value","lags","nobs","1%","5%","10%"]]
adf_df_display.style.format({
    "ADF stat":"{:.3f}",
    "p-value":"{:.4f}",
    "1%":"{:.3f}",
    "5%":"{:.3f}",
    "10%":"{:.3f}"
})


,series,trend,ADF stat,p-value,lags,nobs,1%,5%,10%
0,RGDP_per_capita,ct,-2.205,0.4872,1,180,-4.010,-3.435,-3.142
1,CPIAUCSL,ct,-1.146,0.9209,5,176,-4.011,-3.436,-3.142
2,YGR,c,-14.638,0.0000,0,180,-3.467,-2.878,-2.575
3,INF,c,-4.812,0.0001,4,176,-3.468,-2.878,-2.576


In [ ]:
def coint_summary(y, x, pair_label, trend='c'):
    y = np.log(y.iloc[3:])
    x = np.log(x.iloc[3:])
    tstat, pval, crit = coint(y, x, trend=trend)
    return {
        "Pair": pair_label,
        "Trend": trend,
        "t-stat": tstat,
        "p-value": pval,
        "1%": crit[0],
        "5%": crit[1],
        "10%": crit[2],
        "Decision (5%)": "Reject H₀" if pval < 0.05 else "Fail to Reject H₀"
    }


coint_pairs = {
    "RGDP_per_capita vs CPIAUCSL": ("RGDP_per_capita", "CPIAUCSL", "c"),  # constant
    "Real GDP vs M2": ("GDPC1", "WM2NS", "c")
}

rows = [
    coint_summary(df_q[y], df_q[x], pair_label, trend)
    for pair_label, (y, x, trend) in coint_pairs.items()
]

coint_df = pd.DataFrame(rows)
coint_df_display = coint_df[
    ["Pair","Trend","t-stat","p-value","1%","5%","10%","Decision (5%)"]
]
display(
    coint_df_display.style.format({
        "t-stat":"{:.3f}",
        "p-value":"{:.4f}",
        "1%":"{:.3f}",
        "5%":"{:.3f}",
        "10%":"{:.3f}"
    })
)

,Pair,Trend,t-stat,p-value,1%,5%,10%,Decision (5%)
0,RGDP_per_capita vs CPIAUCSL,c,-2.380,0.3346,-3.959,-3.371,-3.068,Fail to Reject H₀
1,Real GDP vs M2,c,-1.317,0.8246,-3.959,-3.371,-3.068,Fail to Reject H₀
